# The Lattice Boltzmann Method - Multi-Relaxation Time

This notebook uses the Lattice Boltzmann Method (LBM) for numerical simulation of fluid flow to determine and visualize the vortical flow behind a sphere. Written in python and jax. This notebook will focus on using the Multi-Relaxation Time colission operator to enable more stable simulation with the LBM. This particular example increases the Reynolds number to a domain where the BGK collision method would create an unstable simulation.

The vortical flow behind a sphere can have a very complex structure in 3 dimensions. We are going to visualize it using 3D plotting with plotly.
  
The code is adjusted from the the code presented by Machine Learning & Simulation (MLS) in 2D:
- on [Youtube](https://www.youtube.com/watch?v=ZUXmO4hu-20&list=LL&index=1&ab_channel=MachineLearning%26Simulation) and [Github](https://github.com/Ceyron/machine-learning-and-simulation/blob/main/english/simulation_scripts/lattice_boltzmann_method_python_jax.py)

Other code on the LBM:
- LBM in 3D: [Google Colab](https://colab.research.google.com/drive/1F3EH9_2N3lkEpgQXOScR3lcQ6oqCARPk?usp=sharing) and [Github](https://github.com/Ceyron/machine-learning-and-simulation/blob/main/english/simulation_scripts/D3Q19_lattice_bolzmann_method_python_jax.py)

- On stress, force and drag: [Google Colab](https://colab.research.google.com/drive/1oryCdOPXapOWxGSgCDNkvSUQ_MahfoRX?usp=sharing) and [Github](https://github.com/Ceyron/machine-learning-and-simulation/blob/main/english/simulation_scripts/D3Q19_lattice_boltzmann_method_stress_force_drag.py)

- On unit conversion: [Google Colab](https://colab.research.google.com/drive/1OkpFHdGmCEmfEq1a_FgKsiRgc-gh6g2A?usp=sharing)

It is recommended to watch that video first and go through the notebook on 3D LBM, because a lot of explanation of this method, the setup and syntax mentioned in that video and code will be skipped here.

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm import tqdm

# Enable 64bit
jax.config.update("jax_enable_x64", False)

# Setup of the domain
The fluid flow simulation we are preparing is a cylinder in fluid flow, with the aim to make a Karman Vortex Street behind the cylinder. The modelled domain is going to be 300 cells along the x-axis and 50 along the y- and z-axis. The fluid is going to flow into the domain from the left, where x = 0.

In below cell is also the viscosity defined. The viscosity determines the Reynolds number, which is a measure of the characteristics of the flow, and the relaxation $\omega$, which is a relaxation parameter for the LBM method. The inverse ($1/\omega$) is the relaxation time $\tau$.

In [ ]:
circle_radius = 6
ny = 10*circle_radius
nz = 10*circle_radius
nx = 60*circle_radius

reynolds_number = 500
HORIZONTAL_INFLOW_VELOCITY = 0.1          #  cm/s

KINEMATIC_VISCOSITY = (HORIZONTAL_INFLOW_VELOCITY * circle_radius) / reynolds_number
RELAXATION_OMEGA = (1.0 / (3.0 * KINEMATIC_VISCOSITY + 0.5))

print('kinematic viscosity:', KINEMATIC_VISCOSITY, "cm2/s")

The sphere is defined below, as any obstacle used so far: in a boolean array.
To get into the interactive 3D plotting from plotly, we can have a look at our sphere with it:

In [ ]:
x = jnp.arange(nx)
y = jnp.arange(ny)
z = jnp.arange(nz)
X, Y, Z = jnp.meshgrid(x, y, z, indexing="ij")

sphere = jnp.sqrt((X - nx//5)**2 + (Y - ny//2)**2 + (Z - nz//2)**2)
obstacle_mask = sphere < circle_radius

def show_3D_boolean(values, width = 1400, color = 'gray'):
    nx, ny, nz = values.shape
    fig = go.Figure(data=go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=values.flatten(),
        colorscale = [color, color],

        # The plot shows values from 0 to 1. By setting the min to > 0 the domain boundaries don't show up.
        isomin=0.5,

        # Only show the isoplot of specified values (standard is 2, the min and the max)
        caps=dict(x_show=False, y_show=False)
        ))



    # Set the plot size to domain size
    fig.update_layout(
        scene = dict(
            xaxis = dict(range = [0, nx]),
            yaxis = dict(range = [0, ny]),
            zaxis = dict(range = [0, nz]),
            aspectratio = dict(x = nx/nx,
                               y = ny/nx,
                               z = nz/nx)),
        width = width,
        colorscale=dict()
        )

    fig.show()
show_3D_boolean(obstacle_mask)

## D3Q19 - LBM
For the 3D version of the LBM method there is choice between D3Q15, D3Q19 and D3Q27. Here we are going to use the D3Q19 version. There are some differences with the other versions, beyond the different lattice velocities. When taking into account MRT-LBM with the other versions of 3D LBM, check out the relevant lattice velocities and MRT matrices.

For D3Q19 we can define its lattice velocities as usual:

In [ ]:
N_DISCRETE_VELOCITIES = 19

LATTICE_INDICES =     jnp.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18])
LATICE_VELOCITIES_X = jnp.array([0, 1,-1, 0, 0, 0, 0, 1,-1, 1,-1, 1,-1, 1,-1, 0, 0, 0, 0])
LATICE_VELOCITIES_Y = jnp.array([0, 0, 0, 1,-1, 0, 0, 1, 1,-1,-1, 0, 0, 0, 0, 1,-1, 1,-1])
LATICE_VELOCITIES_Z = jnp.array([0, 0, 0, 0, 0, 1,-1, 0, 0, 0, 0, 1, 1,-1,-1, 1, 1,-1,-1])

LATTICE_VELOCITIES = jnp.array([LATICE_VELOCITIES_X,
                                LATICE_VELOCITIES_Y,
                                LATICE_VELOCITIES_Z])

LATTICE_WEIGHTS = jnp.array([# rest particle
                             1/3,

                             # face-connected neighbors
                             1/18, 1/18, 1/18, 1/18, 1/18, 1/18,

                             # edge-connected neighbors
                             1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36, 1/36])

OPPOSITE_LATTICE_INDICES = jnp.array([jnp.where((LATTICE_VELOCITIES.T == -LATTICE_VELOCITIES[:, i]).all(axis = 1))[0] for i in range(N_DISCRETE_VELOCITIES)]).T[0]

RIGHT_VELOCITIES = jnp.where(LATICE_VELOCITIES_X == 1)[0]   # [ 1,  7,  9, 11, 13]
LEFT_VELOCITIES =  jnp.where(LATICE_VELOCITIES_X ==-1)[0]   # [ 2,  8, 10, 12, 14]
YZ_VELOCITIES =    jnp.where(LATICE_VELOCITIES_X == 0)[0]   # [ 0,  3,  4,  5,  6, 15, 16, 17, 18]

## MLT

The evolution equation for the single-relaxation LBM is described as:

$f(c_i, t + 1) = f(c_i, t) - \textbf{S} \left[f(c_i, t) - f^{eq}(c_i, t) \right]$

Where the lefthand side shows the streamed velocities end the righthand side the streaming and collission step. $f(c_i, t + 1)$ are the streamed discrete velocities, $f(c_i, t)$ are the previous discrete velocities and $f^{eq}(c_i, t)$ are the equilibrium version of the previous velocities.
The righthand side shows the collission step with the $\textbf{S}$ and the colission step we are used to. Where in the BGK the relaxation omega $\omega$ (= $\frac{1}{\tau}$) would be, the collission matrix $\textbf{S}$ is shown.
The BGK method is the special case where the matrix $\textbf{S} = \omega I$, where $I$ is the identity matrix.  

But of course, we are not interested in the **single**-relaxation time method (this time)! The single relaxation method is added here to help your understanding of what is to come: the **multi**-relaxation time method. For a more complete explanantion of the MRT method, I would refer you to [this paper](https://www.researchgate.net/publication/7550572_Multiple-Relaxation-Time_Lattice_Boltzmann_Models_in_Three_Dimensions).

The evolution equation for the MRT method is based on the principle of streaming in moment space, rather than lattice velocity space.

This requires the following steps when streaming with the MRT method:

1) Converting to moment space:

> $ m = \textbf{M}\cdot f$

> $ m^{eq} = \textbf{M}\cdot f^{eq}$

2) Streaming in moment space:

> $m_{t + 1} = m_{t} - \textbf{S} \cdot (m_{t} - m^{eq}_{t})$

3) Converting back to velocity space

> $f_{t+1} =  \textbf{M}^{-1}\cdot m_{t+1}$

Which can all be strung together as:

> $f_{t+ 1} = \textbf{M}^{-1}\cdot \textbf{M} \cdot f_{t} - \textbf{M}^{-1}\cdot\textbf{S}\cdot \textbf{M} \cdot \left[f_{t} - f^{eq}_{t} \right]$

And reduced to:

> $f_{t+ 1} =  f_{t} - \hat{\textbf{S}}\cdot \left[f_{t} - f^{eq}_{t} \right]$

by precomputing the collision operator $\hat{\textbf{S}}$:

> $\hat{\textbf{S}} = \textbf{M}^{-1}\cdot\textbf{S}\cdot\textbf{M}$

The matrix $\textbf{M}$ is used for transforming the discrete velocities to the moments $m$. $\textbf{M}^{-1}$ Is the inverse of the transformation matrix $\textbf{M}$. The derivation is complicated and uses Hermite polynomials, but the values for $\textbf{M}$ for D3Q19 (derived using the Gram-Schmidt procedure for those interested) are defined below in the coded cell. The diagonal values for $\hat{\textbf{S}}$ ($s_1$ to $s_{19}$, except for $s_9$ and $s_{13}$) can be chosen between 0 and 2 and tweaked to increase the stability of the model for your boundary conditions and Reynolds number. The values chosen here have been determined to optimal for a different situation described [here](https://ntrs.nasa.gov/api/citations/20000046606/downloads/20000046606.pdf), but suites our purpose here just fine. The values for $s_9$ and $s_{13}$ are $\omega$.

In [ ]:
M = jnp.array([ [  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
                [-30,-11,-11,-11,-11,-11,-11,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8],
                [ 12, -4, -4, -4, -4, -4, -4,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
                [  0,  1, -1,  0,  0,  0,  0,  1, -1,  1, -1,  1, -1,  1, -1,  0,  0,  0,  0],
                [  0, -4,  4,  0,  0,  0,  0,  1, -1,  1, -1,  1, -1,  1, -1,  0,  0,  0,  0],
                [  0,  0,  0,  1, -1,  0,  0,  1,  1, -1, -1,  0,  0,  0,  0,  1, -1,  1, -1],
                [  0,  0,  0, -4,  4,  0,  0,  1,  1, -1, -1,  0,  0,  0,  0,  1, -1,  1, -1],
                [  0,  0,  0,  0,  0,  1, -1,  0,  0,  0,  0,  1,  1, -1, -1,  1,  1, -1, -1],
                [  0,  0,  0,  0,  0, -4,  4,  0,  0,  0,  0,  1,  1, -1, -1,  1,  1, -1, -1],
                [  0,  2,  2, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -2, -2, -2, -2],
                [  0, -4, -4,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1, -2, -2, -2, -2],
                [  0,  0,  0,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1, -1, -1,  0,  0,  0,  0],
                [  0,  0,  0, -2, -2,  2,  2,  1,  1,  1,  1, -1, -1, -1, -1,  0,  0,  0,  0],
                [  0,  0,  0,  0,  0,  0,  0,  1, -1, -1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
                [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, -1, -1,  1],
                [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, -1, -1,  1,  0,  0,  0,  0],
                [  0,  0,  0,  0,  0,  0,  0,  1, -1,  1, -1, -1,  1, -1,  1,  0,  0,  0,  0],
                [  0,  0,  0,  0,  0,  0,  0, -1, -1,  1,  1,  0,  0,  0,  0,  1, -1,  1, -1],
                [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1, -1, -1, -1, -1,  1,  1]])
M_inv = jnp.linalg.inv(M)

s1 = 1.19
s2 = s10 = 1.4
s4 = 1.2
s16 = 1.98
s9 = s13 = RELAXATION_OMEGA

S = jnp.eye(N_DISCRETE_VELOCITIES) * jnp.array([0, s1, s2, 0, s4, 0, s4, 0, s4, s9, s10, s9, s10, s13, s13, s13, s16, s16, s16])

COLLISION_OPERATOR = M_inv @ S @ M

The above definition of $\hat{\textbf{S}}$ can be reduced to the BGK collission method by setting the diagonals of $\hat{\textbf{S}}$ to $\omega$. This will result in an unstable simulation for the boundary conditions set above!

You can see the difference by running the code in below cell (after removing the #):

In [ ]:
# COLLISION_OPERATOR = jnp.eye(N_DISCRETE_VELOCITIES) * RELAXATION_OMEGA

# Useful functions
These functions are independent of the number of dimensions and can run with 2D and 3D simulations.

## Moments
The formulation for the moments is very much the same as for the macroscopic velocities. This is beause it is both a dot-product between a matrix and a vector.


In [ ]:
def get_density(discrete_velocities):
    return jnp.sum(discrete_velocities, axis=-1)

def get_macroscopic_velocities(discrete_velocities, density):
    return jnp.einsum("...Q, dQ -> ...d", discrete_velocities, LATTICE_VELOCITIES) / density[..., jnp.newaxis]

def get_equilibrium_discrete_velocities(macroscopic_velocities, density):
    projected_discrete_velocities = jnp.einsum("dQ, ...d -> ...Q", LATTICE_VELOCITIES, macroscopic_velocities)
    macroscopic_velocity_magnitude = jnp.linalg.norm(macroscopic_velocities, axis=-1, ord=2)
    equilibrium_discrete_velocities = (jnp.einsum("..., Q -> ...Q", density, LATTICE_WEIGHTS) *
        (1 + 3 * projected_discrete_velocities + 9/2 * projected_discrete_velocities**2 -
        3/2 * macroscopic_velocity_magnitude[..., jnp.newaxis]**2))
    return equilibrium_discrete_velocities

def get_moments(discrete_velocities):
    return jnp.einsum('...Q, PQ -> ...P', discrete_velocities, M)


## The steps of the MRT-LBM
The 7 steps as given in the MLS video and code are not adjusted much to the notebook on LBM in 3D. The noteable exception os the collsion operator in the collision step. The same boundary conditions and order of the steps are used.

In [ ]:
@jax.jit
def update(discrete_velocities_prev):
    # (1) Prescribe the outflow BC on the right boundary. Flow can go out, but not back in.
    discrete_velocities_prev = discrete_velocities_prev.at[-1, ..., LEFT_VELOCITIES].set(discrete_velocities_prev[-2, ..., LEFT_VELOCITIES])

    # (2) Determine macroscopic velocities
    density_prev = get_density(discrete_velocities_prev)
    macroscopic_velocities_prev = get_macroscopic_velocities(
        discrete_velocities_prev,
        density_prev)

    # (3) Prescribe Inflow Dirichlet BC using Zou/He scheme in 3D:
    macroscopic_velocities_prev = macroscopic_velocities_prev.at[0, ...].set(VELOCITY_PROFILE[0, ...])
    lateral_densities = get_density(jnp.einsum('i...->...i', discrete_velocities_prev[0, ..., YZ_VELOCITIES]))
    left_densities = get_density(jnp.einsum('i...->...i', discrete_velocities_prev[0, ..., LEFT_VELOCITIES]))
    density_prev = density_prev.at[0, ...].set((lateral_densities + 2 * left_densities) /
                                                (1 - macroscopic_velocities_prev[0, ..., 0]))

    # (4) Compute discrete Equilibria velocities
    equilibrium_discrete_velocities = get_equilibrium_discrete_velocities(
       macroscopic_velocities_prev,
       density_prev)

    # (3) Belongs to the Zou/He scheme
    discrete_velocities_prev =\
          discrete_velocities_prev.at[0, ..., RIGHT_VELOCITIES].set(
              equilibrium_discrete_velocities[0, ..., RIGHT_VELOCITIES])

    # (5) Collide according to MRT
    discrete_velocities_post_collision = (discrete_velocities_prev -
                                          jnp.einsum('...Q, PQ-> ...P',
                                                     discrete_velocities_prev -
                                                     equilibrium_discrete_velocities,
                                          (COLLISION_OPERATOR)))

    # (6) Bounce-Back Boundary Conditions to enfore the no-slip
    for i in range(N_DISCRETE_VELOCITIES):
        discrete_velocities_post_collision = discrete_velocities_post_collision.at[obstacle_mask, LATTICE_INDICES[i]].set(
                                                      discrete_velocities_prev[obstacle_mask, OPPOSITE_LATTICE_INDICES[i]])

    # (7) Stream alongside lattice velocities
    discrete_velocities_streamed = discrete_velocities_post_collision
    for i in range(N_DISCRETE_VELOCITIES):
        discrete_velocities_streamed = discrete_velocities_streamed.at[..., i].set(
            jnp.roll(discrete_velocities_post_collision[..., i],
            LATTICE_VELOCITIES[:, i], axis = (0, 1, 2)))


    return discrete_velocities_streamed

## Ready to run!
Now we define the parameters for plotting and running and innitialize the discrete velocities.

And then we run!

## Results
The images plotted above are already quite beautiful, showing the more complex vorticity compared to the vortex shedding behind a cylinder.  There are different algorithms to determine this, of which the easiest to grasp are the Lambda2 and Q-criterion. Both dependent on determining the strain rate and vorticity tensors.

The Lambda2 method has a [Wikipedia page](https://en.wikipedia.org/wiki/Lambda2_method) that explains the method very well and is reproduced below in the function Lambda2.

The Q-criterion compares the vorticity and strain rate magnitudes and where the vorticity magnitude is larger, considers the node to be in the vortex.

Both methods have a cutoff value to manually tweak for better visualisation.

In [ ]:
PLOT_EVERY_N_STEPS = 1_000
SKIP_FIRST_N_ITERATIONS = 0
N_ITERATIONS = 5_000
noise_magnitude = 0.01
key = jax.random.PRNGKey(0)
VELOCITY_PROFILE = jnp.zeros((nx, ny, nz, 3))
NOISE = jax.random.normal(key, (ny, nz)) * noise_magnitude
VELOCITY_PROFILE = VELOCITY_PROFILE.at[:, :, :, 0].set(HORIZONTAL_INFLOW_VELOCITY + NOISE)
discrete_velocities_prev = get_equilibrium_discrete_velocities(VELOCITY_PROFILE,
                                                               jnp.ones((nx, ny, nz)))

def run(discrete_velocities_prev):
    for i in tqdm(range(N_ITERATIONS)):
        discrete_velocities_next = update(discrete_velocities_prev)
        discrete_velocities_prev = discrete_velocities_next

        if i % PLOT_EVERY_N_STEPS == 0 and i > SKIP_FIRST_N_ITERATIONS - PLOT_EVERY_N_STEPS:
            density = get_density(discrete_velocities_next)
            macroscopic_velocities = get_macroscopic_velocities(
                discrete_velocities_next,
                density)
            velocity_magnitude = jnp.linalg.norm(
                macroscopic_velocities,
                axis=-1,
                ord=2)

            d_u__d_x, d_u__d_y = jnp.gradient(macroscopic_velocities[..., nz//2, 0])
            d_v__d_x, d_v__d_y = jnp.gradient(macroscopic_velocities[..., nz//2, 1])
            curl = (d_u__d_y - d_v__d_x)

            fig = plt.figure(figsize = (15, 3))
            cont = plt.contourf(
                X[:, :, nz//2],
                Y[:, :,  nz//2],
                jnp.flip(velocity_magnitude[:, :,  nz//2], axis = 1),
                cmap="inferno"
                )
            plt.axis('scaled')
            plt.axis('off')
            plt.show()

            fig = plt.figure(figsize = (15, 3))
            cont = plt.contourf(
                X[:, :, nz//2],
                Y[:, :,  nz//2],
                jnp.flip(curl, axis = 1),
                levels = 50,
                cmap="seismic",
                vmin = -0.2, vmax = 0.2)
            plt.axis('scaled')
            plt.axis('off')
            plt.show()

    return discrete_velocities_next

discrete_velocities = run(discrete_velocities_prev)

## Visualizing the vortexes

To really appreciated the complexity, and structures in the vortex, I recommend a look at it in 3D.

But how do you know what is in a vortex? There are many methods to detect where a vortex occurs. I'm highlighting 2 common methods here. The Lambda2 method and the Q-criterion method.

### Lambda2
From the [Wikipedia page](https://en.wikipedia.org/wiki/Lambda2_method):

$\lambda1, \lambda2, \lambda3 = eigh(\mathbf{S}^2 + \mathbf{\Omega}^2)$

Where $\mathbf{S} = \frac{\nabla \mathbf{v} + \nabla \mathbf{v}^T}{2}$, the strain rate tensor and $\mathbf{\Omega} = \frac{\nabla \mathbf{v} - \nabla \mathbf{v}^T}{2}$, the vorticity. The eigh operator returns the eigenvalues $\lambda$.

We require only the second eigenvalue, that's why the name Lambda2. At locations where the $\lambda2$ is below a certain treshold, we decide to call that location a vortex.

### Q-criterion
The Q-criterion also uses the vorticity and the strain rate ([source](https://www.m4-engineering.com/q-criterion-for-vortex-visualization/)):

$Q = |\mathbf{\Omega}|^2 - |\mathbf{S}|^2$

When the Q-value is above a specific treshold value, we decide the vertex to be in a vortex.

In [ ]:
def Lambda2(macroscopic_velocities, negative_cutoff = 0):
  """Use vortex detection algorithm "Lambda2" to get which nodes in your velocity field are inside a vortex.
  """
  # First define the gradients of the velocity field: Δu
  _gradients = jnp.gradient(macroscopic_velocities, axis = (0, 1, 2))
  gradients = jnp.einsum('d...D-> ...dD', jnp.array(_gradients))

  # Get the strain rate tensor: S = (Δu + Δuᵀ)/2
  transposed_gradients = jnp.einsum('...dD -> ...Dd', gradients)
  strain_rate = (gradients + transposed_gradients)/2

  # Get the vorticity tensor: Ω = (Δu - Δuᵀ)/2
  vorticity = (gradients - transposed_gradients)/2

  # Get the eigenvalues (λ) of S² + Ω².
  S2_O2 = strain_rate**2 + vorticity**2
  eigen_values, errors = jnp.linalg.eigh(S2_O2)

  # Select the locations where λ₂ < the negative cutoff. Negative cutoff must be 0 or less.
  in_vortex = (eigen_values < negative_cutoff)[..., 1]
  return in_vortex

def Q(macroscopic_velocities, positive_cutoff = 0):
  """Use vortex detection algorithm "Q-criterion" to get which nodes in your velocity field are inside a vortex.
  """
  # First define the gradients of the velocity field: Δu
  _gradients = jnp.gradient(macroscopic_velocities, axis = (0, 1, 2))
  gradients = jnp.einsum('d...D-> ...dD', jnp.array(_gradients))

  # Get the strain rate tensor: S = (Δu + Δuᵀ)/2
  transposed_gradients = jnp.einsum('...dD -> ...Dd', gradients)
  strain_rate = (gradients + transposed_gradients)/2

  # Get the vorticity tensor: Ω = (Δu - Δuᵀ)/2
  vorticity = (gradients - transposed_gradients)/2

  # Get the vorticity and strain rate magnitude
  n_vorticity = jnp.linalg.norm(vorticity, axis = (-2, -1))
  n_strain_rate = jnp.linalg.norm(strain_rate, axis = (-2, -1))

  # CHeck where the vorticity magnitude is larger than the strain rate magnitude
  Q = (n_vorticity**2 - n_strain_rate**2) > positive_cutoff
  return Q

Now we have the tools to show these vorteces off!

Regretfully, the jnp.linalg.eigh function runs into an Xla error here. However, the Q-criterion works great as well!

I'm keeping the Lambda2 method here as well, for it is often employed and yields better results in other cases I've implemented it.

I'm choosing to have a different cutoff value over the x-axis of the domain. This way we can see the pretty features over more of the domain and reduce noise closer to the sphere.

In [ ]:
density = get_density(discrete_velocities)
macroscopic_velocities = get_macroscopic_velocities(
    discrete_velocities,
    density)

# vortex = Lambda2(macroscopic_velocities, negative_cutoff = -0.00000001)
# vortex = Q(macroscopic_velocities, positive_cutoff = 0.000025)

cutoff = (0.1 + jnp.exp(-(X/10)**2/(nx-1))*4) * 0.000025 # + 1e-6

vortex = Q(macroscopic_velocities, positive_cutoff = cutoff)
show_3D_boolean(vortex)

We can see these gorgeous loops flicking off the sphere. We can also already see the noise that flows into our domain.